# Import Libraries

In [1]:
from swarmintelligence import SlimeMouldAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/SMA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_iteration': 100, 'z': 0.3}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
sma_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                sma_evaluation_results['image_name'].append(image_name)
                sma_evaluation_results['thresholds'].append(threshold)
                sma_evaluation_results['fitness_function'].append(fitness_function)
                sma_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    sma = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    slimeMoulds, best_thresholds = sma.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(sma.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(sma.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                sma_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                sma_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                sma_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                sma_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                sma_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                sma_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                sma_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                sma_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                sma_evaluation_results['Regions'].append(mean_regions)
                sma_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                sma_evaluation_results['Fitness'].append(fitness_values)
                sma_evaluation_results['MSE'].append(list_mse)
                sma_evaluation_results['RMSE'].append(list_rmse)
                sma_evaluation_results['PSNR'].append(list_psnr)
                sma_evaluation_results['SSIM'].append(list_ssim)
                sma_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1900.544172651057
1935.1427497029517
1760.029831850508
1929.4264968514403
1852.371111097189
1933.4480526149869
1917.7294614338884
1909.8964077189949
1941.6708735644106
1853.9923916076216
1864.9124278503612
1929.7076812309629
1945.2387645147073
1938.163608392143
1891.287967334211
1837.5406516974124
1845.0510194572962
1933.4278670048725
1878.8450731877294
1937.0230212680178
1855.3416901887763
1936.8273446114451
1941.0138514355845
1893.3427369188248
1940.0967438812672
1908.133464609219
1863.1343261019297
1827.1252097261524
1933.6495913237427
1945.212968322338
Mean Fitness = 1899.3109186050017
	 AirplaneF16.tiff | Threshold =  3
1926.4988105772024
1965.3128016055907
1958.4265485908745
1976.3472776895896
1984.776274258874
1973.2069746254174
1936.9755700935498
1945.0769555569714
1998.8047294153491
1964.902033512852
2000.3819081820996
2016.151012967876
1941.381467296088
1930.804817700139
1976.100695071168
1926.6544721074142
2018.4761356751005
1996.844288713

3187.1759232361537
3062.6956614191504
Mean Fitness = 3077.446665267694
	 Mandrill.tiff | Threshold =  2
1533.299716292733
1527.4664941362125
1215.2449765407737
1467.9875040559887
1527.8867841105737
1467.9004699488091
1218.6802721605616
1494.0338371101557
1488.5203627080584
1413.4484070143017
1457.4703464462477
1398.8501075138238
1461.416161404
1244.1151391145474
1499.8245714925774
1308.0146268589197
1443.3793666822125
1463.5122807543003
1541.1834658898952
1460.8040984837253
1470.6748078346113
1533.296075049494
1470.0165181095394
1377.9430205562892
1523.6012339681474
1520.9877282774632
1501.8336261530671
1539.7362544455518
1431.2624645107176
1529.9941422036509
Mean Fitness = 1451.079495327565
	 Mandrill.tiff | Threshold =  3
1548.2045087546921
1544.2453668517715
1588.1396070376682
1500.3019048126612
1609.2632022907746
1544.1216259683902
1592.244993560328
1568.3293275785495
1473.5068473246683
1540.2133735796465
1600.8321485107845
1380.5825854430157
1533.0659122799395
1544.4657605929078
1

4170.992977185749
4053.7566296741807
4109.948931288429
4099.08676573795
4103.931893447294
4048.347440927068
4125.901250364987
4084.666970975183
Mean Fitness = 4107.98036278307


## Visualize Results using DataFrame

In [7]:
sma_evaluation_results_df =pd.DataFrame(sma_evaluation_results)
sma_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[111, 177]",0.605190,1899.310919,2301.335755,47.687533,14.620033,0.757553,0.935879,"[[111, 255, 255, 255, 255, 255, 255, 255, 255,...","[0.869579553604126, 0.6486091613769531, 0.6466...","[1900.544172651057, 1935.1427497029517, 1760.0...","[2662.768295288086, 2332.6821060180664, 3183.7...","[51.60201832572139, 48.29784784043763, 56.4247...","[13.877469835599639, 14.452248029610079, 13.10...","[0.770413198786015, 0.7896660521395991, 0.7451...","[0.9274181308282241, 0.9384325429497612, 0.910..."
1,AirplaneF16.tiff,3,otsu,max,"[78, 134, 180]",1.270497,1968.045379,2011.156348,44.544639,15.222233,0.765535,0.946810,"[[78, 255, 255, 255, 180, 255, 255, 255, 255, ...","[1.1703214645385742, 1.3367934226989746, 1.351...","[1926.4988105772024, 1965.3128016055907, 1958....","[1797.8604469299316, 1955.7278137207031, 2378....","[42.40118449913789, 44.22361149567845, 48.7738...","[15.58324382859142, 15.217719486701984, 14.367...","[0.7168045878970921, 0.7613769935015016, 0.783...","[0.9463448227654281, 0.9438729985570365, 0.937..."
2,AirplaneF16.tiff,4,otsu,max,"[57, 113, 161, 195]",1.349807,2012.155471,1458.234763,37.290891,16.926691,0.769647,0.959427,"[[113, 255, 195, 195, 195, 195, 195, 195, 195,...","[1.3328173160552979, 1.323824405670166, 1.1633...","[1964.1964392493105, 2038.7159311323653, 1995....","[2229.967071533203, 1907.3327293395996, 703.04...","[47.22252716165457, 43.67302061158124, 26.5149...","[14.647819107152705, 15.326538996460396, 19.66...","[0.7957092869915807, 0.8108431661042719, 0.780...","[0.9424470833747326, 0.953653498430446, 0.9697..."
3,AirplaneF16.tiff,5,otsu,max,"[36, 90, 132, 171, 202]",1.483900,2037.178439,1220.264876,33.951938,17.788569,0.771306,0.965496,"[[90, 202, 202, 202, 202, 202, 202, 202, 202, ...","[1.440394401550293, 1.451077938079834, 1.41336...","[2068.36148832426, 2012.6468608757748, 2017.40...","[1254.9122200012207, 1942.2183837890625, 654.9...","[35.42474022489397, 44.070606800781206, 25.592...","[17.144670125029435, 15.247823003102177, 19.96...","[0.7338844608637243, 0.7654392615847073, 0.782...","[0.9671804345733586, 0.9486722619864457, 0.973..."
4,Lena.png,2,otsu,max,"[97, 155]",1.301174,1860.228170,3403.611285,57.357374,13.102709,0.592233,0.861417,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.6040899753570557, 1.3118226528167725, 1.331...","[1782.0183025287588, 1756.5897334463357, 1918....","[2241.573257446289, 4796.551422119141, 3029.21...","[47.3452559127764, 69.25713986383744, 55.03832...","[14.62527424141766, 11.321512561205271, 13.317...","[0.5903768678939875, 0.5444107220530676, 0.612...","[0.866587353016657, 0.8366451667555517, 0.8869..."
5,Lena.png,3,otsu,max,"[60, 120, 169]",1.326767,1971.928951,2446.315512,48.191295,14.695740,0.636809,0.894288,"[[169, 169, 169, 169, 169, 169, 169, 169, 169,...","[1.4753773212432861, 1.1127941608428955, 1.383...","[2052.36679646283, 2066.0955813620185, 1936.96...","[1629.4883308410645, 1826.8483238220215, 3421....","[40.36692124550824, 42.74164624604464, 58.4893...","[16.01029106143139, 15.513778698161998, 12.789...","[0.6879034322971914, 0.6578765413295549, 0.585...","[0.919145754387529, 0.916743653965417, 0.86919..."
6,Lena.png,4,otsu,max,"[55, 97, 141, 186]",1.375566,2062.142592,1544.170349,38.159091,16.738075,0.682299,0.922032,"[[186, 186, 186, 186, 186, 186, 186, 186, 186,...","[1.110795021057129, 1.4248650074005127, 1.4093...","[2123.8583405282634, 1923.9302945759018, 1980....","[956.2814254760742, 1549.8041152954102, 1755.3...","[30.92380030779002, 39.36755155321969, 41.8966...","[18.324946406156766, 16.22803551099106, 15.687...","[0.7070193492264457, 0.6526293134330234, 0.659...","[0.9395196885370424, 0.8902109715979009, 0.913..."
7,Lena.png,5,otsu,max,"[38, 75, 122, 158, 199]",1.406406,2092.731120,

# Save results

In [8]:
pickle.dump(sma_evaluation_results_df, open('results/evaluation/SMA/SMA_otsu_df.pkl', 'wb'))
pickle.dump(sma_evaluation_results, open('results/evaluation/SMA/SMA_otsu_dict.pkl', 'wb'))